In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import matplotlib as mpl

from pathlib import Path
from torch import cdist
import os
import glob

In [14]:
from catboost import CatBoostRegressor
from hmmlearn import hmm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import umap
import math
# from statsmodels.tsa.arima.model import ARIMA

/Users/dashtbozorgi/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import EncoderFactory
from DatasetManager import DatasetManager

In [2]:
ROOT_DIR = Path().cwd()
while not ROOT_DIR.joinpath("data").exists():
    ROOT_DIR = ROOT_DIR.parent
os.chdir(ROOT_DIR)

In [3]:
case_id_col = "Identification No."
timestamp_col = "Time (hrs)"
work_day_col = "Working Day (d)"
target_col = "Product Content pre clean up (g/L)"

In [4]:
# 95 bioreactors are 5L, 11 are 50L or higher

In [5]:
cols_to_keep = [case_id_col, timestamp_col, target_col, work_day_col, 'BR Temp (°C)', 'pH (Internal)', 'Viability (%)', 'VCD (10^6 cells/mL)', 
                'Glutamine (mmol/L)', 'Glutamate (mmol/L)', 'Glucose (g/L)','Lactate (g/L)', 'NH4+ (mmol/L)', 
                'Na+ (mmol/L)', 'K+ (mmol/L)', 'Ca2+ (g/L)']

features = ['BR Temp (°C)', 'pH (Internal)', 'Viability (%)', 'VCD (10^6 cells/mL)', 
                'Glutamine (g/L)', 'Glutamate (g/L)', 'Glucose (g/L)','Lactate (g/L)', 'NH4+ (g/L)', 
                'Na+ (g/L)', 'K+ (g/L)', 'Ca2+ (g/L)']

In [ ]:
# ambr_path = Path("./data/Original/From_Mohammad")
# ambr_files = glob.glob("./data/Original/From_Mohammad/*.csv*")
astra_path = Path("./data/Original")
astra_files = glob.glob("./data/Original/*.xlsx*")
len(astra_files)

In [ ]:
selected_file = astra_files[0]
selected_file

In [ ]:
first_file = pd.ExcelFile(selected_file)
df = first_file.parse("Main Analysis Data Base ", header=1)
# first_timestamp = first[first['working day']==0]['Date & Time'].unique()[0]

In [ ]:
df = df[cols_to_keep]

In [ ]:
missing_threshold = 0.7
uids_with_less_missing = {}

for column in df.columns:
    missing_percentage = df[column].isnull().sum() / len(df)
    if missing_percentage > missing_threshold:
        uids_with_less_missing[column] = df[df[column].notnull()]['UID'].unique().tolist()

uids_with_less_missing

In [ ]:
null_columns_dict = {}

for uid in df[case_id_col].unique():
    uid_data = df[df[case_id_col] == uid]
    null_columns = []
    
    for column in uid_data.columns:
        if uid_data[column].isnull().all():
            null_columns.append(column)
    
    if null_columns:
        null_columns_dict[uid] = null_columns



In [ ]:
for key, value in null_columns_dict.items():
    print(key, value)


In [ ]:
keys_with_titer = []

for key, value in null_columns_dict.items():
    if target_col in value:
        keys_with_titer.append(key)


In [ ]:
df = df[~df[case_id_col].isin(keys_with_titer)]

In [ ]:
df[case_id_col].nunique()

In [ ]:
num_unique_keys = len(null_columns_dict.keys())
print(num_unique_keys)

In [ ]:
df = df[df[work_day_col]>=0]

In [ ]:
df[work_day_col] = df[work_day_col].astype(int)


In [ ]:
df[work_day_col] = df[case_id_col].str.split('_').str[1]


In [ ]:
grouped_df = df.groupby(case_id_col)
df = grouped_df.filter(lambda x: len(x) >= 5)


In [ ]:
null_rows = df[df.isnull().any(axis=1)]
null_rows

In [ ]:


# Group the DataFrame by case ID
grouped_df = df.groupby(case_id_col)

# Find groups where the target column is always 0
groups_with_target_0 = grouped_df.filter(lambda x: (x[target_col] == 0).all())

# Print the groups where the target column is always 0
for case_id, group in groups_with_target_0.groupby('Identification No.'):
    print(f"Case ID: {case_id}")
    # print(group)
    print()

# Remove the groups where the target column is always 0
df = df.drop(groups_with_target_0.index)


In [ ]:
# Group data by case ID
grouped_df = df.groupby('Identification No.')

# Iterate over each group
for group_name, group_data in grouped_df:
    # Check if the column is always null in the group
    for col in group_data.columns:
        if group_data[col].isnull().any():
            print("Group Name:", group_name)
            print("Null Column:", col)
            print()

print('finished')


In [ ]:
df[case_id_col].nunique()

In [ ]:
df = df.drop_duplicates(subset=[case_id_col, work_day_col], keep="last")

In [ ]:
def impute_with_closest(group):
    # Sort the DataFrame by working day
    group = group.sort_values(work_day_col, ascending=True)

    # Forward fill missing values based on working day
    group = group.fillna(method='ffill')

    # Backward fill any remaining missing values
    group = group.fillna(method='bfill')
    return group

In [ ]:
def impute_with_median(group):
    return group.fillna(group.median())

In [ ]:
df = df.groupby(case_id_col).apply(impute_with_closest)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.isnull().any()


In [ ]:
# Define the molar mass of the substance
molar_mass_glutamine = 146.14  
molar_mass_glutamate = 147.13

# Convert mmol/L to g/L
df['Glutamine (g/L)'] = df['Glutamine (mmol/L)'] * molar_mass_glutamine /1000
df['Glutamate (g/L)'] = df['Glutamate (mmol/L)'] * molar_mass_glutamate /1000

In [ ]:
# Define the molar mass of the substance
molar_mass_sodium = 22.99 
molar_mass_potassium = 39.10
molar_mass_calcium = 40.08

# Convert mmol/L to g/L
df['Na+ (g/L)'] = df['Na+ (mmol/L)'] * molar_mass_sodium /1000
df['K+ (g/L)'] = df['K+ (mmol/L)'] * molar_mass_potassium /1000
# df['Calcium (g/L)'] = df['Calcium (mmol/L)'] * molar_mass_calcium /1000

In [ ]:
molar_mass_ammonia = 17.031

df['NH4+ (g/L)'] = df['NH4+ (mmol/L)'] * molar_mass_ammonia /1000

In [ ]:
df.drop(['Glutamine (mmol/L)', 'Glutamate (mmol/L)', 'Na+ (mmol/L)', 'K+ (mmol/L)', 'NH4+ (mmol/L)'], axis=1, inplace=True)

In [ ]:
df.to_csv('data/processed/CSL312_preprocessed_forTiter_notNormalised_6Feb25.csv', index=False)

In [ ]:
df = df.sort_values(by=[case_id_col, work_day_col], ascending=True)

In [ ]:
df.columns

##############################################################################

In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, PowerTransformer, Normalizer

# Sample data
data = df[features]  # Replace with your actual data
# scaler = StandardScaler()
# scaler = MinMaxScaler()
# scaler = RobustScaler()
# scaler = MaxAbsScaler()
scaler = PowerTransformer()
# scaler = Normalizer()
# scaler = TimeSeriesScalerMinMax()
X_scaled = scaler.fit_transform(data)

# Fit nearest neighbors
k = 5  # Choose k as min_samples for DBSCAN
neigh = NearestNeighbors(n_neighbors=k)
nbrs = neigh.fit(X_scaled)
distances, indices = nbrs.kneighbors(X_scaled)

# Sort and plot distances to the k-th nearest neighbor
distances = np.sort(distances[:, k-1])
plt.plot(distances)
plt.ylabel('k-distance')
plt.xlabel('Data points sorted by distance')
plt.title('K-Distance Graph - PowerTransformer')
# plt.rcParams['xtick.labelsize'] = 0.1
plt.tick_params(axis='both', which='major', labelsize=16)
plt.grid(True)
plt.show()

In [ ]:
X_scaled = df[features]

In [ ]:
from sklearn.cluster import DBSCAN
import umap

reducer = umap.UMAP(random_state = 123)
embedding = reducer.fit_transform(X_scaled)
embedding.shape

In [ ]:
import joblib

In [ ]:
joblib.dump(reducer, 'CSL312_reducer.pkl')

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score

In [ ]:
# Sample data
X_scaled = embedding

# Define a range of eps values to test
eps_values = np.linspace(0.1, 5.0, 10)

# List to store the silhouette scores
silhouette_scores = []

# Compute silhouette scores for different eps values
for eps in eps_values:
    db = DBSCAN(eps=eps, min_samples=20).fit(X_scaled)
    labels = db.labels_
    # Ignore the silhouette score if only one cluster or all points are noise
    if len(set(labels)) > 1:
        score = silhouette_score(X_scaled, labels)
        # score = calinski_harabasz_score(X_scaled, labels)
        silhouette_scores.append(score)
    else:
        silhouette_scores.append(-1)  # Invalid score

# Plot silhouette scores
plt.plot(eps_values, silhouette_scores)
# plt.ylabel('Silhouette Score')
plt.ylabel('Silhouette score')
plt.xlabel('eps value')
plt.xticks(np.arange(0, 6, step=0.5))
# plt.rcParams['xtick.labelsize'] = 0.7
plt.tick_params(axis='both', which='major', labelsize=12)
plt.title('Silhouette score vs. eps - embedding', fontsize=16)
plt.show()

In [ ]:
eps = 1.5
min_samples = 30
dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(X_scaled)
dbscan.labels_

In [ ]:
pd.Series(dbscan.labels_).value_counts()

In [ ]:
# Visualize the dbscan clustered data
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=dbscan.labels_, cmap='Accent', edgecolors='k')
# plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], marker='X', s=100, c='red', label='Centroids')
plt.title(f'Clustered Data with DBSCAN - UMAP Embedding, eps={eps}', fontsize=16)
# plt.xlabel("Lactate (g/L)")
# plt.ylabel("Glucose (g/L)")
plt.tick_params(axis='both', which='major', labelsize=12)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=12)
# plt.legend()
plt.show()

In [ ]:
df['Cluster'] = dbscan.labels_

In [ ]:
import random

my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Calculate the number of elements to select
num_elements = int(len(my_list) * 0.7)

# Select 70% of elements randomly
selected_elements = random.sample(my_list, num_elements)

print(selected_elements)


In [ ]:
def split_data(data, train_ratio=0.70, split="temporal", leave_out=[], seed=22):  
    # split into train and test using temporal split

    data[work_day_col] = data[work_day_col].astype(int)
    print(data[work_day_col].dtype)
    grouped = data.groupby(case_id_col)
    # start_timestamps = grouped[timestamp_col].min().reset_index()
    if split == "temporal":
        start_timestamps = start_timestamps.sort_values(timestamp_col, ascending=True, kind="mergesort")
        train_ids = list(start_timestamps[case_id_col])[:int(train_ratio*len(start_timestamps))]

    elif split == "temporal sim":
        # Calculate the number of elements to select
        num_elements = int(len(data[case_id_col].unique()) * train_ratio)
        train_ids = random.sample(list(data[case_id_col].unique()), num_elements)

    elif split == "random":
        np.random.seed(seed)
        start_timestamps = start_timestamps.reindex(np.random.permutation(start_timestamps.index))
        train_ids = list(start_timestamps[case_id_col])[:int(train_ratio*len(start_timestamps))]

    elif split == "loo":
        try:
            leave_id = str(leave_out[0])
            all_ids = list(data[case_id_col].unique())
            train_ids = all_ids
            train_ids.remove(leave_id)

        except IndexError:
            print("Leave out list is empty.")
            
    train = data[data[case_id_col].isin(train_ids)].sort_values(work_day_col, ascending=True, kind='mergesort')
    test = data[~data[case_id_col].isin(train_ids)].sort_values(work_day_col, ascending=True, kind='mergesort')

    return (train, test)

In [ ]:
def get_size_max_len(sequences):
    # Tokenize the words
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequences)
    vocab_size = len(tokenizer.word_index) + 1

    # Convert words to sequences of integers
    sequences_int = tokenizer.texts_to_sequences(sequences)

    # Generate input and target sequences for training
    max_sequence_len = max([len(seq) for seq in sequences_int])
    
    return vocab_size, max_sequence_len

In [ ]:
def tokenize_and_pad(sequences, max_sequence_len):
    # Tokenize the words
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequences)
    vocab_size = len(tokenizer.word_index) + 1
    
    # Convert words to sequences of integers
    sequences_int = tokenizer.texts_to_sequences(sequences)
    
    # Generate input and target sequences for training
    # max_sequence_len = max([len(seq) for seq in sequences_int])
    input_sequences = []
    target_sequences = []
    for seq in sequences_int:
        # print(seq)
        for i in range(1, len(seq)):
            input_seq = seq[:i]
            target_seq = seq[i]
            input_sequences.append(input_seq)
            target_sequences.append(target_seq)
            
    # Pad sequences to have the same length
    input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
    target_sequences = np.array(target_sequences)
    
    return input_sequences, target_sequences

In [ ]:
def remove_first_row(group):
    return group.iloc[1:]

def remove_last_row(group):
    return group.iloc[:-1]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.python.framework.errors_impl import InvalidArgumentError
import tensorflow as tf

from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from hmmlearn import hmm
import math
# from statsmodels.tsa.arima.model import ARIMA

In [ ]:
df_normalized = df.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Define the columns to normalize
columns_to_normalize = features + [target_col]

scalers = {}
# Create a StandardScaler object
scaler = StandardScaler()

for column in columns_to_normalize:
    scalers[column] = StandardScaler()
    df_normalized[column] = scalers[column].fit_transform(df_normalized[[column]])

# Normalize the selected columns
# df_normalized[columns_to_normalize] = scaler.fit_transform(df_normalized[columns_to_normalize])

In [ ]:
df_normalized['Titer (g/L) original'] = scalers[target_col].inverse_transform(df_normalized[target_col].to_numpy().reshape(-1, 1))

In [ ]:
df_normalized['Titer (g/L) original'].describe()

In [ ]:
df_normalized.to_csv('data/processed/CSL312_preprocessed_forTiter_normalised_6Feb25.csv', index=False)

In [6]:
df_normalized = pd.read_csv('data/processed/CSL312_preprocessed_forTiter_normalised_6Feb25.csv')

In [7]:
len(features)

12

In [8]:
UIDs = df_normalized[case_id_col].unique()

In [9]:
results = []
method = 'Catboost'
config = 'no_encoding_no_bucketing'
# config = 'no_encoding_bucketing'
# config = 'encoding_no_bucketing'
# config = 'encoding_bucketing'

In [12]:
from processors import processor_factory
# from data.dataset_confs import dataset_configs

# config = dataset_configs['CSL_5L']
use_encoding = False
use_bucketing = False
num_nearest_neighbors = 100
distance_metric = 'euclidean'

# processor = CSL_5LProcessor("5L", use_encoding, use_bucketing, num_nearest_neighbors, distance_metric)
processor = processor_factory.get_processor('CSL312', use_encoding, use_bucketing, num_nearest_neighbors, distance_metric)

In [18]:
# Define the values to experiment with
num_nearest_neighbors_values = [100, 200, 300, 400, 500]
distance_metrics = ['euclidean_expotential', 'cosine_expotential', 'dtw_exponential']

# Loop through the values and create processors
for num_nearest_neighbors in num_nearest_neighbors_values:
    for distance_metric in distance_metrics:
        print(f"Experimenting with num_nearest_neighbors={num_nearest_neighbors} and distance_metric={distance_metric}")
        
        # Create the processor with the current values
        processor = processor_factory.get_processor('CSL312', use_encoding=False, use_bucketing=False, num_nearest_neighbors=num_nearest_neighbors, distance_metric=distance_metric)
        
    

        data = df_normalized.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort')
        data['Target'] = data.groupby(case_id_col)[target_col].shift(-1)
        # data['Target_orig'] = data.groupby(case_id_col)['Titer (g/L) original'].shift(-1)
        data['Target'] = data.groupby(case_id_col)['Target'].ffill().bfill()
        # data['Target_orig'] = data.groupby(case_id_col)['Target_orig'].ffill()

        historic, current = processor.split_data(data, train_ratio=0.5, split="temporal sim")
        historic.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)
        current.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)

        if config == 'no_encoding_bucketing' or config == 'encoding_bucketing':
            features_used = features + ['Cluster']
        else:
            features_used = features

        # n_neighbors = 200
        # # Initialize the NearestNeighbors model
        # nn_model = NearestNeighbors(n_neighbors=n_neighbors, metric="cosine")
        # # Fit the model on the historic data
        # nn_model.fit(historic[features_used])

        batch_size = 50

        nn_model = processor.train_nn_model(historic[features_used])

        for start in range(0, len(current), batch_size):
            end = start + batch_size
            batch = current.iloc[start:end]
            # Find the n nearest neighbors for the selected row
            # distances, indices = nn_model.kneighbors([row[features_used]])
            distances, indices = processor.find_nearest_neighbors(nn_model, batch[features_used])
            nearest_neighbors = pd.concat([historic.iloc[indices[i]] for i in range(len(batch))])

            target = nearest_neighbors['Target'].values
            target_test = batch['Target']

            if target_test is None:
                continue
            

            if method == 'Catboost':
                # Create the CatBoostRegressor model
                model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='MAE', verbose=0)
                model.fit(nearest_neighbors[features_used], target)

            if method == 'HMM':
                # Create an instance of the HMM model
                model = hmm.GaussianHMM(n_components=7)  # Specify the number of hidden states
                model.fit(nearest_neighbors[features_used])

            # Make predictions on the testing data
            preds = model.predict(batch[features_used])

            # true_conc_glu = row['Target_orig']
            # preds_scaled = scalers[target_col].inverse_transform(preds.reshape(-1, 1))

            batch.loc[:, 'predicted_value'] = preds

            results.append(batch)

            # Add the current row with its prediction to the historic data
            # row_with_prediction[target_col] = preds_scaled[0][0]
            historic = pd.concat([historic, batch], ignore_index=True)
            historic.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)
            nn_model =processor.train_nn_model(historic[features_used])  # Refit the model with the updated historic data

            results_df = pd.concat(results)

            # Calculate metrics
            true_values = results_df[target_col]
            predicted_values = results_df['predicted_value']

            MAE_t = mean_absolute_error(true_values, predicted_values)
            MSE_t = mean_squared_error(true_values, predicted_values)
            RMSE_t = math.sqrt(MSE_t)
            r2_t = r2_score(true_values, predicted_values)
            mape_t = mean_absolute_percentage_error(true_values, predicted_values)

            # Save results to a CSV file
            # results_df.to_csv('predictions_with_row_data.csv', index=False)

            # Print metrics
            print(f"MAE: {MAE_t}")
            print(f"MSE: {MSE_t}")
            print(f"RMSE: {RMSE_t}")
            print(f"R2: {r2_t}")
            print(f"MAPE: {mape_t}")

            results_df.to_csv(f'results/CSL312_{method}_{config}_{num_nearest_neighbors}_{distance_metric}.csv', index=False)


Experimenting with num_nearest_neighbors=100 and distance_metric=cosine
int64


/var/folders/sb/50cj9frx6576wh5j6rbpnb6c0000gp/T/ipykernel_36568/1086096361.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch.loc[:, 'predicted_value'] = preds


MAE: 0.2563861127545148
MSE: 0.16054375317751737
RMSE: 0.4006791149754593
R2: 0.836241682166444
MAPE: 1.5117903735243223


/var/folders/sb/50cj9frx6576wh5j6rbpnb6c0000gp/T/ipykernel_36568/1086096361.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch.loc[:, 'predicted_value'] = preds


MAE: 0.2549395796571474
MSE: 0.15899919111520172
RMSE: 0.39874702646565496
R2: 0.8365735200937883
MAPE: 1.497471556747054


...

MAE: 0.25243800860629095
MSE: 0.15410130659343987
RMSE: 0.3925573927382337
R2: 0.8452204291527264
MAPE: 1.6089106797744621


/var/folders/sb/50cj9frx6576wh5j6rbpnb6c0000gp/T/ipykernel_36568/1086096361.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch.loc[:, 'predicted_value'] = preds


MAE: 0.2523736804687545
MSE: 0.15392985476405813
RMSE: 0.39233895392129764
R2: 0.8454288535279606
MAPE: 1.6077488091611838


/var/folders/sb/50cj9frx6576wh5j6rbpnb6c0000gp/T/ipykernel_36568/1086096361.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch.loc[:, 'predicted_value'] = preds


MAE: 0.2523162259967348
MSE: 0.15385795821617854
RMSE: 0.39224731766600845
R2: 0.8455696873540627
MAPE: 1.6090009140609685


In [ ]:
results = []
method = 'Catboost'
# config = 'no_encoding_no_bucketing'
config = 'no_encoding_bucketing'
# config = 'encoding_no_bucketing'
# config = 'encoding_bucketing'

data = df_normalized.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort')
data['Target'] = data.groupby(case_id_col)[target_col].shift(-1)
data['Target_orig'] = data.groupby(case_id_col)['Titer (g/L) original'].shift(-1)
data['Target'] = data.groupby(case_id_col)['Target'].ffill().bfill()
data['Target_orig'] = data.groupby(case_id_col)['Target_orig'].ffill().bfill()

historic, current = split_data(data, train_ratio=0.5, split="temporal sim")
historic.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)
current.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)

if config == 'no_encoding_bucketing' or config == 'encoding_bucketing':
    features_used = features + ['Cluster']
else:
    features_used = features

n_neighbors = 200
# Initialize the NearestNeighbors model
nn_model = NearestNeighbors(n_neighbors=n_neighbors)
# Fit the model on the historic data
nn_model.fit(historic[features_used])

for index, row in current.iterrows():
    # Find the n nearest neighbors for the selected row
    distances, indices = nn_model.kneighbors([row[features_used]])
    nearest_neighbors = historic.iloc[indices[0]]

    target = nearest_neighbors['Target'].values
    target_test = row['Target']

    if target_test is None:
        continue
    

    if method == 'Catboost':
        # Create the CatBoostRegressor model
        model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='MAE', verbose=0)
        model.fit(nearest_neighbors[features_used], target)

    if method == 'HMM':
        # Create an instance of the HMM model
        model = hmm.GaussianHMM(n_components=7)  # Specify the number of hidden states
        model.fit(df_without_last[features+['Cluster']])

    # Make predictions on the testing data
    preds = model.predict(row[features_used])

    true_conc_glu = row['Target_orig']
    preds_scaled = scalers[target_col].inverse_transform(preds.reshape(-1, 1))
    
    key = row[case_id_col] + '_' + str(row[work_day_col])
    results.append({
        'key': key,
        'row_data': row.to_dict(),
        'true_value': true_conc_glu,
        'predicted_value': preds_scaled[0][0]
    })

    # Add the current row with its prediction to the historic data
    row_with_prediction = row.copy()
    # row_with_prediction[target_col] = preds_scaled[0][0]
    historic = pd.concat([historic, pd.DataFrame([row_with_prediction])], ignore_index=True)
    historic.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)
    nn_model.fit(historic[features_used])  # Refit the model with the updated historic data


In [ ]:
results_df = pd.DataFrame(results)

# Calculate metrics
true_values = results_df['true_value']
predicted_values = results_df['predicted_value']

MAE_t = mean_absolute_error(true_values, predicted_values)
MSE_t = mean_squared_error(true_values, predicted_values)
RMSE_t = math.sqrt(MSE_t)
r2_t = r2_score(true_values, predicted_values)
mape_t = mean_absolute_percentage_error(true_values, predicted_values)

# Save results to a CSV file
# results_df.to_csv('predictions_with_row_data.csv', index=False)

# Print metrics
print(f"MAE: {MAE_t}")
print(f"MSE: {MSE_t}")
print(f"RMSE: {RMSE_t}")
print(f"R2: {r2_t}")
print(f"MAPE: {mape_t}")

In [ ]:
results_df.to_csv(f'results/CSL312_{method}_{config}_200_results.csv', index=False)

In [ ]:
import importlib
import sys
import time

In [ ]:
min_prefix_length = 1
max_prefix_length = df_normalized[work_day_col].max()

In [ ]:
# Remove both EncoderFactory and AggregateTransformer from sys.modules
modules_to_remove = ['EncoderFactory', 'transformers.AggregateTransformer', 'DatasetManager', "dataset_confs"]
for module in modules_to_remove:
    if module in sys.modules:
        del sys.modules[module]

# Re-import the modules
import EncoderFactory
import transformers.AggregateTransformer
import DatasetManager
from DatasetManager import DatasetManager

# Reload the modules
importlib.reload(transformers.AggregateTransformer)
importlib.reload(EncoderFactory)
# importlib.reload(DatasetManager)

In [ ]:
dataset_manager = DatasetManager("CSL312")

In [ ]:
start_test_prefix_generation = time.time()
dt_prefixes = dataset_manager.generate_prefix_data(df_normalized, min_prefix_length, max_prefix_length)
test_prefix_generation_time = time.time() - start_test_prefix_generation

In [ ]:
dt_prefixes[target_col].isnull().sum()

In [ ]:
dt_prefixes

In [ ]:
encoder = EncoderFactory.get_encoder(method='agg', case_id_col=case_id_col, static_cat_cols=None, static_num_cols=None, dynamic_cat_cols=['Cluster'],
                dynamic_num_cols=features, target_cols=[target_col,'Titer (g/L) original'], work_day_col=work_day_col, fillna=True, max_events=None, activity_col=None, resource_col=None, timestamp_col=timestamp_col,
                scale_model=None)

In [ ]:
dt_transformed = encoder.transform(dt_prefixes)
dt_transformed.columns

In [ ]:
dt_transformed = dt_transformed.reset_index(drop=False, inplace=False)
dt_transformed[work_day_col] = dt_transformed[case_id_col].str.split('_').str[2]
dt_transformed[work_day_col] = dt_transformed[work_day_col].fillna(0)
dt_transformed[case_id_col] = dt_transformed[case_id_col].str.split('_').str[0] + '_' + dt_transformed[case_id_col].str.split('_').str[1]


In [ ]:

dt_transformed


In [ ]:
features_new = [col for col in dt_transformed.columns if col not in ["Cluster", "'Titer (g/L) original'", case_id_col, target_col, work_day_col]]

In [ ]:
results = []
method = 'Catboost'
# config = 'no_encoding_no_bucketing'
# config = 'no_encoding_bucketing'
config = 'encoding_no_bucketing'
# config = 'encoding_bucketing'

data = dt_prefixes.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort')
# data['Target'] = data.groupby(case_id_col)[target_col].shift(-1)
# data['Target_orig'] = data.groupby(case_id_col)['Titer (g/L) original'].shift(-1)
# data['Target'] = data.groupby(case_id_col)['Target'].ffill()
# data['Target_orig'] = data.groupby(case_id_col)['Target_orig'].ffill()

historic, current = split_data(data, train_ratio=0.5, split="temporal sim")
historic.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)
current.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)

if config == 'no_encoding_bucketing' or config == 'encoding_bucketing':
    features_used = features + ['Cluster']
else:
    features_used = features

# Initialize the NearestNeighbors model
nn_model = NearestNeighbors(n_neighbors=200)
# Fit the model on the historic data
nn_model.fit(historic[features_used])

for index, row in current.iterrows():
    # Find the n nearest neighbors for the selected row
    distances, indices = nn_model.kneighbors([row[features_used]])
    nearest_neighbors = historic.iloc[indices[0]]

    target = nearest_neighbors['Target'].values
    target_test = row['Target']

    if target_test is None:
        continue
    

    if method == 'Catboost':
        # Create the CatBoostRegressor model
        model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='MAE', verbose=0)
        model.fit(nearest_neighbors[features_used], target)

    if method == 'HMM':
        # Create an instance of the HMM model
        model = hmm.GaussianHMM(n_components=7)  # Specify the number of hidden states
        model.fit(df_without_last[features+['Cluster']])

    # Make predictions on the testing data
    preds = model.predict(row[features_used])

    true_conc_glu = row['Target_orig']
    preds_scaled = scalers[target_col].inverse_transform(preds.reshape(-1, 1))
    
    key = row[case_id_col] + '_' + str(row[work_day_col])
    results.append({
        'key': key,
        'row_data': row.to_dict(),
        'true_value': true_conc_glu,
        'predicted_value': preds_scaled[0][0]
    })

    # Add the current row with its prediction to the historic data
    row_with_prediction = row.copy()
    # row_with_prediction[target_col] = preds_scaled[0][0]
    historic = pd.concat([historic, pd.DataFrame([row_with_prediction])], ignore_index=True)
    historic.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)
    nn_model.fit(historic[features_used])  # Refit the model with the updated historic data


In [ ]:
results_df = pd.DataFrame(results)

# Calculate metrics
true_values = results_df['true_value']
predicted_values = results_df['predicted_value']

MAE_t = mean_absolute_error(true_values, predicted_values)
MSE_t = mean_squared_error(true_values, predicted_values)
RMSE_t = math.sqrt(MSE_t)
r2_t = r2_score(true_values, predicted_values)
mape_t = mean_absolute_percentage_error(true_values, predicted_values)

# Save results to a CSV file
# results_df.to_csv('predictions_with_row_data.csv', index=False)

# Print metrics
print(f"MAE: {MAE_t}")
print(f"MSE: {MSE_t}")
print(f"RMSE: {RMSE_t}")
print(f"R2: {r2_t}")
print(f"MAPE: {mape_t}")

In [ ]:
results_df.to_csv(f'results/CSL312_{method}_{config}_results.csv', index=False)

In [ ]:
results_df = pd.DataFrame(results)

# Calculate metrics
true_values = results_df['true_value']
predicted_values = results_df['predicted_value']

MAE_t = mean_absolute_error(true_values, predicted_values)
MSE_t = mean_squared_error(true_values, predicted_values)
RMSE_t = math.sqrt(MSE_t)
r2_t = r2_score(true_values, predicted_values)
mape_t = mean_absolute_percentage_error(true_values, predicted_values)

# Save results to a CSV file
# results_df.to_csv('predictions_with_row_data.csv', index=False)

# Print metrics
print(f"MAE: {MAE_t}")
print(f"MSE: {MSE_t}")
print(f"RMSE: {RMSE_t}")
print(f"R2: {r2_t}")
print(f"MAPE: {mape_t}")

In [ ]:
results_df

#################################################################################################

In [ ]:
results = []
method = 'Catboost'
config = 'no_encoding_no_bucketing'
# config = 'no_encoding_bucketing'
# config = 'encoding_no_bucketing'
# config = 'encoding_bucketing'

data = df_normalized.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort')
data['Target'] = data.groupby(case_id_col)[target_col].shift(-1)
data['Target_orig'] = data.groupby(case_id_col)['Titer (g/L) original'].shift(-1)
data['Target'] = data.groupby(case_id_col)['Target'].ffill().bfill()
data['Target_orig'] = data.groupby(case_id_col)['Target_orig'].ffill().bfill()

historic, current = split_data(data, train_ratio=0.5, split="temporal sim")
historic.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)
current.sort_values([case_id_col, work_day_col], ascending=True, kind='mergesort', inplace=True)

if config == 'no_encoding_bucketing' or config == 'encoding_bucketing':
    features_used = features + ['Cluster']
else:
    features_used = features


for index, row in current.iterrows():

    target = historic['Target'].values
    target_test = row['Target']

    if target_test is None:
        continue
    

    if method == 'Catboost':
        # Create the CatBoostRegressor model
        model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='MAE', verbose=0, random_state=123)
        model.fit(historic[features_used], target)

    if method == 'HMM':
        # Create an instance of the HMM model
        model = hmm.GaussianHMM(n_components=7)  # Specify the number of hidden states
        model.fit(df_without_last[features+['Cluster']])

    # Make predictions on the testing data
    preds = model.predict(row[features_used])

    true_conc_glu = row['Target_orig']
    preds_scaled = scalers[target_col].inverse_transform(preds.reshape(-1, 1))
    
    key = row[case_id_col] + '_' + str(row[work_day_col])
    results.append({
        'key': key,
        'row_data': row.to_dict(),
        'true_value': true_conc_glu,
        'predicted_value': preds_scaled[0][0]
    })

In [ ]:
results_df = pd.DataFrame(results)

# Calculate metrics
true_values = results_df['true_value']
predicted_values = results_df['predicted_value']

MAE_t = mean_absolute_error(true_values, predicted_values)
MSE_t = mean_squared_error(true_values, predicted_values)
RMSE_t = math.sqrt(MSE_t)
r2_t = r2_score(true_values, predicted_values)
mape_t = mean_absolute_percentage_error(true_values, predicted_values)

# Save results to a CSV file
# results_df.to_csv('predictions_with_row_data.csv', index=False)

# Print metrics
print(f"MAE: {MAE_t}")
print(f"MSE: {MSE_t}")
print(f"RMSE: {RMSE_t}")
print(f"R2: {r2_t}")
print(f"MAPE: {mape_t}")

In [ ]:
results_df.to_csv(f'results/CSL312_baseline_{method}_{config}_results.csv', index=False)

In [ ]:
results_df_400 = pd.read_csv('results/CSL312_Catboost_no_encoding_bucketing_cosine_results.csv')
results_df_300 = pd.read_csv('results/CSL312_Catboost_no_encoding_bucketing_chebyshev_results.csv')
results_df_200 = pd.read_csv('results/CSL312_Catboost_no_encoding_bucketing_200_results.csv')
results_df_100 = pd.read_csv('results/CSL312_Catboost_no_encoding_bucketing_DTW_results.csv')

In [ ]:
# Extract the row_data column
row_data_df = pd.json_normalize(results_df_400['row_data'])

# Drop the original row_data column from results_df_baseline
tmp = results_df_400.drop(columns=['row_data'])

# Concatenate the normalized row_data DataFrame with the original DataFrame
tmp = pd.concat([tmp, row_data_df], axis=1)

# Retain true_value and predicted_value columns
tmp = tmp[['true_value', 'predicted_value'] + list(row_data_df.columns)]

In [ ]:
# Extract the row_data column
row_data_df = pd.json_normalize(results_df_300['row_data'])

# Drop the original row_data column from results_df_baseline
tmp2 = results_df_300.drop(columns=['row_data'])

# Concatenate the normalized row_data DataFrame with the original DataFrame
tmp2 = pd.concat([tmp2, row_data_df], axis=1)

# Retain true_value and predicted_value columns
tmp2 = tmp2[['true_value', 'predicted_value'] + list(row_data_df.columns)]

In [ ]:
# Extract the row_data column
row_data_df = pd.json_normalize(results_df_200['row_data'])

# Drop the original row_data column from results_df_baseline
tmp3 = results_df_200.drop(columns=['row_data'])

# Concatenate the normalized row_data DataFrame with the original DataFrame
tmp3 = pd.concat([tmp3, row_data_df], axis=1)

# Retain true_value and predicted_value columns
tmp3 = tmp3[['true_value', 'predicted_value'] + list(row_data_df.columns)]

In [ ]:
# Extract the row_data column
row_data_df = pd.json_normalize(results_df_100['row_data'])

# Drop the original row_data column from results_df_baseline
tmp4 = results_df_100.drop(columns=['row_data'])

# Concatenate the normalized row_data DataFrame with the original DataFrame
tmp4 = pd.concat([tmp4, row_data_df], axis=1)

# Retain true_value and predicted_value columns
tmp4 = tmp4[['true_value', 'predicted_value'] + list(row_data_df.columns)]

In [ ]:
def calculate_moving_avg_mae(df, true_col, pred_col, window_size=5):
    true_values = df[true_col].to_numpy()
    predicted_values = df[pred_col].to_numpy()

    num_rows_list = []
    mae_list = []

    for i in range(2, len(true_values) + 1):
        num_rows_list.append(i)
        mae = mean_absolute_error(true_values[:i], predicted_values[:i])
        mae_list.append(mae)

    mae_df = pd.DataFrame({'num_rows': num_rows_list, 'mae': mae_list})
    mae_df['moving_avg_mae'] = mae_df['mae'].rolling(window=window_size).mean()
    
    return mae_df

# Calculate moving average MAE for tmp
mae_df_tmp = calculate_moving_avg_mae(tmp, 'true_value', 'predicted_value')

# Calculate moving average MAE for tmp2
mae_df_tmp2 = calculate_moving_avg_mae(tmp2, 'true_value', 'predicted_value')

# Calculate moving average MAE for tmp2
mae_df_tmp3 = calculate_moving_avg_mae(tmp3, 'true_value', 'predicted_value')

# Calculate moving average MAE for tmp2
mae_df_tmp4 = calculate_moving_avg_mae(tmp4, 'true_value', 'predicted_value')

# Plot the moving average MAE for both DataFrames
plt.figure(figsize=(10, 6))
plt.plot(mae_df_tmp['num_rows'], mae_df_tmp['moving_avg_mae'], label='cosine', color='blue')
plt.plot(mae_df_tmp2['num_rows'], mae_df_tmp2['moving_avg_mae'], label='levenshtein', color='red')
plt.plot(mae_df_tmp3['num_rows'], mae_df_tmp3['moving_avg_mae'], label='euclidean', color='green')
plt.plot(mae_df_tmp4['num_rows'], mae_df_tmp4['moving_avg_mae'], label='DTW', color='black')
plt.xlabel('Number of Observed Rows')
plt.xlabel('Number of Observed Rows')
plt.ylabel('Moving Average MAE')
plt.title('CSL312')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_results = pd.DataFrame(results).T
# catboost regular 
pd.options.display.float_format = '{:.2f}'.format
df_results[df_results.columns].mean()

In [ ]:
df_results = pd.DataFrame(results).T
# HMM regular 
pd.options.display.float_format = '{:.2f}'.format
df_results[df_results.columns].mean()